# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Exploring data

In [8]:
file = 'event_datafile_new.csv'

In [9]:
import pandas as pd
df = pd.read_csv(file)

In [10]:
df.loc[df['sessionId'] == 338]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Table music_history

In [11]:
create_table = """
CREATE TABLE IF NOT EXISTS music_history (
artist_name text,
song_title text,
song_length decimal, 
session_id int,
item_in_session int, 
PRIMARY KEY (session_id
            ,item_in_session)
) 
"""

In [12]:
try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# order artist 	firstName 	gender 	itemInSession 	lastName 	length 	level 	location 	sessionId 	song 	userId
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history(artist_name, song_title, song_length, session_id, item_in_session)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
SELECT artist_name,
    song_title,
    song_length 
FROM music_history 
WHERE session_id = 338 
  AND item_in_session = 4 
"""

In [15]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Table music_history_by_user

In [16]:
create_table = """
CREATE TABLE IF NOT EXISTS music_history_by_user (
artist_name text,
song_title text,
user_first_name text,
user_last_name text,
user_id int,
session_id int,
item_in_session int, 
PRIMARY KEY (user_id
            ,session_id
            ,item_in_session)
) 
"""

In [17]:
try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# order artist 	firstName 	gender 	itemInSession 	lastName 	length 	level 	location 	sessionId 	song 	userId
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_user(artist_name, song_title, user_first_name, user_last_name, user_id, session_id, item_in_session)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[10], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [19]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query="""
SELECT artist_name,
    song_title,
    user_first_name,
    user_last_name
FROM music_history_by_user
WHERE user_id = 10
  AND session_id = 182
"""              

In [20]:
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone 10 Sylvie Cruz
Three Drives 10 Sylvie Cruz
Sebastien Tellier 10 Sylvie Cruz
Lonnie Gordon 10 Sylvie Cruz


In [21]:
df.loc[(df['userId'] == 10) & (df['sessionId'] == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


## Table music_history_by_song

In [22]:
create_table = """
CREATE TABLE IF NOT EXISTS music_history_by_song (
song_title text,
user_first_name text,
user_last_name text,
user_id int,
PRIMARY KEY (song_title,
             user_id)
) 
"""

In [23]:
try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# order artist 	firstName 	gender 	itemInSession 	lastName 	length 	level 	location 	sessionId 	song 	userId
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_song(song_title, user_first_name, user_last_name, user_id)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [25]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
SELECT user_first_name,
    user_last_name
FROM music_history_by_song
WHERE song_title = 'All Hands Against His Own'
"""

In [26]:
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [27]:
df.loc[df['song'] == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [28]:
## TO-DO: Drop the table before closing out the sessions

In [29]:
drop_tables = ["""DROP TABLE IF EXISTS music_history""", """DROP TABLE IF EXISTS music_history_by_user""","""DROP TABLE IF EXISTS music_history_by_song"""]

In [30]:
try:
    for table in drop_tables:
        session.execute(table)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()